# *Spaceship Titanic*

#  <div style="color: #F66;">setting</div>

In [ ]:
# データ処理、可視化系
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-darkgrid')

# 機械学習系
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
# 欠損値補完
from sklearn.impute import SimpleImputer
# KFold
from sklearn.model_selection import StratifiedKFold
# 評価指標
from sklearn.metrics import accuracy_score
# 機械学習モデル
import lightgbm as lgb

# メモリ管理、warning
import gc
import warnings

warnings.filterwarnings('ignore')

# <div style="color: #F66;">データの読み込み</div>

In [ ]:
%%time
# pd.read_csvでデータの読み込み
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')
ss = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

# 長さを取得しておくと後で便利
TRAIN_LEN = len(train)

# サンプルサイズの確認
print(f'sample size\n{"="*20}')
print(f'train: {train.shape}')
print(f'test: {test.shape}')
print(f'\nrunning time\n{"="*20}')

# <div style="color: #f66;">**EDA**</div>

### <div style="color: #f66;">概要確認</div>

In [ ]:
# 最初の5行を見る
train.head(5)

In [ ]:
test.head(5)

### <div style="color: #F66;">特徴量(列)の説明</div>


* `PassengerId` - 各乗客の一意のID。各Idはggggg_ppの形式をとり、gggggはその乗客が一緒に旅行しているグループを、ppはそのグループ内の番号を表します。グループ内の人は家族であることが多いが、必ずしもそうではない。


* `HomePlanet` - 乗客が出発した惑星、通常は定住している惑星。


* `CryoSleep` - 乗客が航海中、仮死状態になることを選択したかどうかを示す。冷凍睡眠中のお客様は、キャビンに閉じ込められます。


* `Cabin` - 乗客が滞在しているキャビンの番号。deck/num/sideの形式で、sideはP（左舷）かS（右舷）のどちらかである。


* `Destination` - 乗客が下船する惑星。


* `Age` - 乗客の年齢。


* `VIP` - 乗客が航海中に特別なVIPサービスを受けたかどうか。


* `RoomService`, `FoodCourt`, `Spa`, `VRDeck` - 宇宙船タイタニックの多くの豪華な設備で乗客が請求した金額です。


* `Name` - 乗客の姓と名。


* `Transported` - 乗客が別の次元に輸送されたかどうか。これは、ターゲット、あなたが予測しようとしている列です。

### <div style="color: #F66;">要約情報の確認</div>

In [ ]:
train.info()

In [ ]:
test.info()

* `train`, `test`ともに欠損値があることが確認できる

### <div style="color: #F66;">要約統計量の確認</div>

In [ ]:
train.describe()

### <div style="color: #F66;">欠損値の確認</div>

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

* 大体データの2.5%が欠損値だとわかる
* 正直これくらいならdropしてもよさそう

### <div style="color: #F66;">Target(`Transported`)の分布</div>

In [ ]:
print(train['Transported'].value_counts())
train['Transported'].value_counts().plot.bar();

* ほぼ同量である

## <div style="color: #F66;">**特徴量のtype別にデータを見る**</div>

In [ ]:
print(f'要約\n{"="*20}')
print(train.info())
print('\n\n')
print(f'特徴量ごとのユニークな変数の数\n{"="*20}')
print(train.nunique())
print('\n\n')
train.head()

* `HomePlanet`, `CryoSleep`, `Destination`, `VIP` はカテゴリ特徴量
* `PassengerId`, `Cabin`, `Name`　はテキストデータ
* `RoomService`, `FoodCourt`, `Spa`, `VRDeck` は連続した数値

In [ ]:
# 特徴量を整理する

FEATURES = list(test.columns)
TARGET = 'Transported'
# カテゴリ特徴量
cat_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

# テキストデータ
text_features = ['PassengerId', 'Cabin', 'Name']

# 連続した数値
continuous_features = ['Age', 'RoomService', 'FoodCourt', 'Spa', 'VRDeck']

* データの分布を見るにあたり、`train`, `test`を結合して全体の分布を見る
* 結合した`DataFrame`を`df`とする。また、`train`, `test`の識別を可能にするために`type`列に明記しておく

In [ ]:
# 元のdfはそのままにしておきたいのでcopyしておく
train_copy = train.copy()
test_copy = test.copy()

# trainとtestを結合
df = pd.concat([train_copy.iloc[:, :-1], test_copy], ignore_index=True, axis=0)

# メモリ確保のためにコピーは消去
del train_copy, test_copy
gc.collect()

# 確認
df

## <div style="color: #F66;">・カテゴリ特徴量 `cat_features`</div>

In [ ]:
# 全体の分布を確認
nrows = 2
ncols = 2
i = 0
fig, axes = plt.subplots(nrows, ncols, figsize=(20,10))
for nrow in range(nrows):
    for ncol in range(ncols):
        cat_feature = cat_features[i]
        sns.countplot(x=df[cat_feature], ax=axes[nrow, ncol])
        axes[nrow, ncol].set_xlabel(cat_feature, fontsize=20)
        i += 1

In [ ]:
# trainデータのみで分布を確認、TARGETごとにした
nrows = 2
ncols = 2
i = 0
fig, axes = plt.subplots(nrows, ncols, figsize=(20,10))
for nrow in range(nrows):
    for ncol in range(ncols):
        cat_feature = cat_features[i]
        sns.countplot(x=train[cat_feature], ax=axes[nrow, ncol], hue=train[TARGET], palette=sns.color_palette('Set1'))
        axes[nrow, ncol].set_xlabel(cat_feature, fontsize=20)
        i += 1

* `HomePlanet`ではEuropaがTrue、EarthがFalseの傾向を見られる
* `CryoSleep`がTrueのときTrue, FalseのときFalseの傾向が見られる

## <div style="color: #F66;">・　連続数値特徴量 `continuos_features`</div>

* 連続型変数はとりあえず箱ひげ図を確認してみる

In [ ]:
# 全体でバイオリンプロットを作成
fig, axes = plt.subplots(1, 5, figsize=(25, 8))
for i, continuous_feature in enumerate(continuous_features):
    sns.violinplot(data=df[continuous_feature], ax=axes[i])
    axes[i].set_xlabel(continuous_feature, fontsize=18)

* `Age`以外は結構えぐい分布している
* `RoomService`,`FoodCourt`,`Spa`,`VRDeck`は0が多いように見える
* `Age`は25~30にかけてが多い

In [ ]:
# trainのTARGETで分けて確認
nrows = 2
ncols = 2
i = 0
fig, axes = plt.subplots(nrows, ncols, figsize=(22,15))
fig.subplots_adjust(hspace=0.4, wspace=0.2)
for nrow in range(nrows):
    for ncol in range(ncols):
        continous_feature = continuous_features[i]
        axes[nrow, ncol].set_title(continous_feature, fontsize=20)
        sns.violinplot(data=train, x=TARGET, y=continous_feature, ax=axes[nrow, ncol])
        axes[nrow, ncol].set_ylabel('How used', fontsize=10)        
        axes[nrow, ncol].set_xlabel(TARGET, fontsize=15)
        i += 1

* 結局多くの人が下に集中しているので分かりにくい,,,

In [ ]:
# 0の人数を抽出する
for continous_feature in continuous_features:
    if continous_feature == 'Age':
        continue
    print(f'train[{continous_feature}]が0の人')
    print(f'{"="*20}')
    print((train[continous_feature]==0).value_counts())
    print()

* 0の人が多いので、0を除いて考える

In [ ]:
# trainのTARGETで分けて確認
# ただし、0の人は除く
nrows = 2
ncols = 2
i = 0
fig, axes = plt.subplots(nrows, ncols, figsize=(22,15))
fig.subplots_adjust(hspace=0.4, wspace=0.2)
for nrow in range(nrows):
    for ncol in range(ncols):
        continous_feature = continuous_features[i]
        axes[nrow, ncol].set_title(continous_feature, fontsize=20)
        sns.violinplot(data=train[train[continous_feature]>0], x=TARGET, y=continous_feature, ax=axes[nrow, ncol])
        axes[nrow, ncol].set_ylabel('How used', fontsize=10)       
        axes[nrow, ncol].set_xlabel(TARGET, fontsize=15)
        i += 1

* 多少見やすくなった
* `FoodCourt`だけ分布の形状が少し異なる？

## <div style="color: #F66;">`RoomService`,`FoodCourt`,`Spa`,`VRDeck`を購入フラグで分割する</div>

In [ ]:
for continous_feature in [cf for cf in continuous_features if cf != 'Age']:
    df[f'use_{continous_feature}'] = np.where(df[continous_feature] > 0, 1, 0)
df.head()    

## <div style="color: #F66;">・ テキスト特徴量 `text_features`</div>

In [ ]:
df[text_features]


* `PassengerId`からは家族の有無が読み取れる
* `Cabin`は`/`で区切ることでデータを細分化できる
* `Name`　は難しい…

## <div style="color: #F66;">テキスト特徴量から新しい特徴量を作る</div>

### <div style="color: #F66;">・ `PassengerId`から家族の有無のフラグを作る</div>

In [ ]:
# 'group_id'_'personal_id'というふうに構成されている

# group_idからなるリストを作成
group_id_list = [group_id.split('_')[0] for group_id in df['PassengerId'].to_list()]

# group_idに重複があれば家族がいると判定する
has_group = [True if group_id_list.count(group_id) >1 else False for group_id in group_id_list]

# dfに追加
df['has_group'] = has_group
# 可視化用にデータの更新
train['has_group'] = has_group[:TRAIN_LEN]

df

* 家族フラグを追加できた
* 一応分布も見ておく

In [ ]:
# 分布の確認
plt.figure(figsize=(10, 7))
sns.countplot(data=train, x='has_group', hue=TARGET, palette='Set1')
plt.xlabel('has_group', fontsize=20)
plt.legend(loc='upper center', fontsize=17);

* グループできた人の方がTrueになる傾向がある
* いい感じの特徴量が生成できた

## <div style="color: #F66;">・ `Cabin`を分割する</div>

In [ ]:
# Cabinは'deck'/'num'/'side'となっている
cabin_list = df['Cabin']


cabin_deck_list, cabin_num_list, cabin_side_list = [], [], []
for cabin in cabin_list:
    # nanの処理
    if pd.isna(cabin):
        cabin_deck_list.append(np.nan)
        cabin_num_list.append(np.nan)
        cabin_side_list.append(np.nan)
    # 分割してそれぞれ追加
    else:
        cabin_deck, cabin_num, cabin_side = cabin.split('/')
        cabin_deck_list.append(cabin_deck)
        cabin_num_list.append(int(cabin_num))
        cabin_side_list.append(cabin_side)       

# dfに追加
df['Cabin_deck'] = cabin_deck_list
df['Cabin_num'] = cabin_num_list
df['Cabin_side'] = cabin_side_list

# 可視化用にtrainも更新
train['Cabin_deck'] = cabin_deck_list[:TRAIN_LEN]
train['Cabin_num'] = cabin_num_list[:TRAIN_LEN]
train['Cabin_side'] = cabin_side_list[:TRAIN_LEN]

df

* cabinを要素ごとに分割できた
* 各要素ごとに可視化する

In [ ]:
# ユニークな数の確認
cabin_components = ['Cabin_deck','Cabin_num', 'Cabin_side']
df[cabin_components].nunique()

* `Cabin_deck`, `Cabin_side`はユニークな数が少ない、ヒストグラムで比較
* `Cabin_num` は数が多い、散布図で見てみる

In [ ]:
# 分布の確認
fig, axes = plt.subplots(3, figsize=(15, 30))
fig.subplots_adjust(hspace=0.3)

sns.countplot(data=train, x='Cabin_deck', hue=TARGET, ax=axes[0], palette='Set1', order=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'])
axes[0].legend(fontsize=20)
axes[0].set_xlabel('Cabin_deck', fontsize=20)

sns.countplot(data=train, x='Cabin_side', hue=TARGET, ax=axes[1], palette='Set2')
axes[1].legend(fontsize=20)
axes[1].set_xlabel('Cabin_side', fontsize=20)

sns.violinplot(data=train, x=TARGET, y='Cabin_num', axes=[2], palette='Set3')
axes[2].set_ylabel('Cabin_num', fontsize=20)
axes[2].set_xlabel(TARGET, fontsize=20);

* `Cabin_deck`がB, Cの人はTrueになる確率が高い
* `Cabin_side`はSの方がTrueになる確率が高い
* `Cabin_num`は特に情報は得られない。強いていうならFalseの方が値がわずかに大きい傾向がある

## <div style="color: #F66;">・　`Name`を分割する</div>

In [ ]:
# Nameは'given_name' 'family_name'というように構成されている
given_name_list, family_name_list = [], []
for name in df['Name'].tolist():
    # Nanの処理
    if pd.isna(name):
        given_name_list.append(np.nan)
        family_name_list.append(np.nan)        
        
    else:
        given_name, family_name = name.split(' ')
        given_name_list.append(given_name)
        family_name_list.append(family_name)
    
df['Given_Name'] = given_name_list
df['Family_Name'] = family_name_list
train['Given_Name'] = given_name_list[:TRAIN_LEN]
train['Family_Name'] = family_name_list[:TRAIN_LEN]
df.head()

In [ ]:
# ユニークな数の名前を調べる
print(f'df.shape: {df.shape}\n')
df[['Given_Name', 'Family_Name']].nunique()

* 6人に一人の割合で同じ姓・名であることがわかる
* 降順ソートして見てみる

In [ ]:
# TrueのGiven_Nameのみを抽出、ソートして50個グラフにする
given_true = train[train[TARGET]==True]['Given_Name'].value_counts(ascending=False)[:50]
fig, ax = plt.subplots(figsize=(20, 7))
sns.barplot(x=given_true.index, y=given_true.values, palette='cool', ax=ax)
ax.set_title('Given_Name(Tranported=True)')
ax.tick_params(axis="x", labelrotation=45)

del given_true

In [ ]:
# FalseのGiven_Nameのみを抽出、ソートして50個グラフにする
given_false = train[train[TARGET]==False]['Given_Name'].value_counts(ascending=False)[:50]
fig, ax = plt.subplots(figsize=(20, 7))
sns.barplot(x=given_false.index, y=given_false.values, palette='autumn', ax=ax)
ax.set_title('Given_Name(Tranported=False)')
ax.tick_params(axis="x", labelrotation=45)

del given_false

In [ ]:
# TrueのFamily_Nameのみを抽出、ソートして50個グラフにする
family_true = train[train[TARGET]==True]['Family_Name'].value_counts(ascending=False)[:50]
fig, ax = plt.subplots(figsize=(20, 7))
sns.barplot(x=family_true.index, y=family_true.values, palette='winter', ax=ax)
ax.set_title('Family_Name(Tranported=True)')
ax.tick_params(axis="x", labelrotation=45)

del family_true

In [ ]:
# FalseのFamily_Nameのみを抽出、ソートして50個グラフにする
family_false = train[train[TARGET]==False]['Family_Name'].value_counts(ascending=False)[:50]
fig, ax = plt.subplots(figsize=(20, 7))
sns.barplot(x=family_false.index, y=family_false.values, palette='spring', ax=ax)
ax.set_title('Family_Name(Tranported=False)')
ax.tick_params(axis="x", labelrotation=45)

del family_false

* 名前による影響はわからない
* 特に使えなさそうだからdropしていいかも

# <div style="color: #F66;">**前処理**</div>

## <div style="color: #F66;">Label Encoding</div>

In [ ]:
# dtypeがobjectかつ、ユニークな要素数が10未満なカテゴリを抽出する
cat_features = [cat_col for cat_col in df.select_dtypes(include='object').columns if df[cat_col].nunique() < 10]

# 欠損値つきでencodingしたいからceを採用
ce_ord = ce.ordinal.OrdinalEncoder(cols=cat_features, handle_missing='return_nan')
df = ce_ord.fit_transform(df)
# dtypeを変換してわかりやすくする
df = df.astype({cat_col: 'category' for cat_col in cat_features})

# TargetもEncodingする。後でdecodeする
le_target = LabelEncoder()
target = pd.Series(le_target.fit_transform(train[TARGET]))
display(df.head(), target)

## <div style="color: #F66;">欠損値処理</div>

In [ ]:
print(f'欠損値の内訳\n{"="*20}')
print(df.isnull().sum())

print(f'\n全ての列が欠損値の行の合計\n{"="*20}')
print(df.isnull().all(axis=1).sum())

print(f'\n１つでも欠損値を含む行の合計\n{"="*20}')
print(df.isnull().any(axis=1).sum())

* 流石に全行dropはよくない
* `category`は最頻値で埋める。`float`はかなり歪みがある分布をしているので、中央値で補完する

In [ ]:
category_features = df.select_dtypes('category').columns
float_features = df.select_dtypes('float').columns

# 比較ようにdfを分ける
df_filled = df.copy()
# カテゴリ特徴量は最頻値埋め (なぜかfillnaが使えないのでsklearnでやる)
mode_imputer = SimpleImputer(strategy='most_frequent')
df_filled[category_features] = mode_imputer.fit_transform(df[category_features])

# 数値は中央値埋め
df_filled[float_features] = df[float_features].fillna(df[float_features].median())

print(df_filled.isnull().sum())
df_filled.head()

* とりあえず欠損値補完できた
* この欠損値補完あんまり良くないかもしれない… → 精度が低下したから

## <div style="color: #F66;">特徴量を選択する</div>

In [ ]:
# もう一度特徴量を確認する
df.info(), df.columns

In [ ]:
# ベースとなる特徴量
base_feature = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# 作成した特徴量を使う
extend_feature = base_feature + ['has_group', 'Cabin_deck' ,'Cabin_side']
add_useflag_ex_feature = extend_feature + ['use_RoomService', 'use_FoodCourt', 'use_Spa', 'use_VRDeck']

In [ ]:
# データフレームを色々作る
df_dict_list = [
    # {'desc': dfの説明, 'df': 学習するdf}
    {'desc': 'カテゴリ追加:なし, 欠損値: 未処理', 'df': df[base_feature][:TRAIN_LEN]},
    {'desc': 'カテゴリ追加:あり, 欠損値: 未処理', 'df': df[extend_feature][:TRAIN_LEN]},
    {'desc': 'カテゴリ追加:なし, 欠損値: 補完済', 'df': df_filled[base_feature][:TRAIN_LEN]},
    {'desc': 'カテゴリ追加:あり, 欠損値: 補完済', 'df': df_filled[extend_feature][:TRAIN_LEN]},
    {'desc': 'カテゴリ追加:あり(使用フラグ), 欠損値: 未処理', 'df': df[add_useflag_ex_feature][:TRAIN_LEN]},    
    {'desc': 'カテゴリ追加:あり(使用フラグ), 欠損値: 補完済', 'df': df_filled[add_useflag_ex_feature][:TRAIN_LEN]}
]

# <div style="color: #F66;">**機械学習**</div>

## <div style="color: #F66;">モデル作成,CV</div>

In [ ]:
# KFoldの分割数
N_SPLIT = 10
# 再現性、比較の妥当性のために設定
RANDOM_STATE = 373

# パラメータの設定
params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'max_depth': 4,             # depthは小さくする　→ 可視化のため
        'learning_rate': 0.005,      # 学習量を多くした分、rateは下げる
        'num_leaves': 11,           # 0.7*max_depth^2
        'n_estimators': 10000,       # early_stoppingで高めに設定。10000だと割とストレス
        'random_state': RANDOM_STATE
}

# 関数定義
def lgb_cv(desc, df):
    """
    与えられたデータに対してクロスバリデーションを行い、その結果を集計する
    
    Parameters
    ----------
    desc : string
        対象のデータフレームの説明
        
    df : pd.DataFrame
        分析対象のデータフレーム
        
    Returns
    ----------
    fold_scores_df : pd.DataFrame
        AccuracyをFoldごとにデータフレームにしたもの
    
    fimps_df : pd.DataFrame
        FeatureImportanceをデータフレームにしたもの
        
    """
    
    # returnの格納用
    fold_scores = {}
    fimps = pd.DataFrame()

    # 説明変数、目的変数
    X = df
    y = target
    
    # クロスバリデーションを作成
    cv = StratifiedKFold(n_splits=N_SPLIT, shuffle=True, random_state=RANDOM_STATE)
    # Fold回数とtrain,testのindexを取得
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        
        # train,testにデータを分割
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # モデルを作成
        lgbm = lgb.LGBMClassifier(**params)
        lgbm.fit(X_train, y_train,
                  eval_metric='binary_logloss',  # early_stoppingの評価指標(学習用の'metric'パラメータにも同じ指標が自動入力される)
                  eval_set=[(X_test, y_test)],
                  callbacks=[lgb.early_stopping(stopping_rounds=60, verbose=0)]) # early_stopping用コールバック関数 verboseは説明みたいなやつ

        # 予測値を生成
        y_pred = lgbm.predict(X_test)
        # スコアを計算、格納
        score = accuracy_score(y_test, y_pred)
        fold_scores.update({f'KFold={idx+1}': score})
        # 特徴量重要度をデータフレームにする
        fimp = pd.DataFrame(data=lgbm.booster_.feature_importance(importance_type='gain'), 
                            index=X.columns, 
                            columns=[f'{idx+1}_importance'])
        # データフレームを拡張
        fimps = pd.concat([fimps, fimp], axis=1)
    
    # 平均のaccuracyを加える
    fold_scores.update({'accuracy of mean': np.mean(list(fold_scores.values()))})
    
    print(f'df: {desc}\n処理が完了しました。\n')
    # returnする
    return pd.DataFrame(fold_scores, index=[desc]).T, fimps

In [ ]:
fold_scores_dfs = []
fimps_dfs = []

# 全てのdfでループする
for df_dict in df_dict_list:
    fold_scores_df, fimps_df = lgb_cv(**df_dict)
    # スコアをまとめる
    fold_scores_dfs.append(fold_scores_df)
    # 特徴量重要度をまとめる
    fimps_dfs.append(fimps_df)

## <div style="color: #F66;">データごとの差の確認</div>

In [ ]:
# dfごとのスコアを比較
pd.concat(fold_scores_dfs, axis=1)

* 色々なパラメータを試したが、カテゴリ追加あり・欠損値未処理が一番精度が高かった。反対に、カテゴリ追加なし・欠損値補完済が一番精度が悪かった。

* あまり差がつかない？ (最大で0.01ほどしか変化がない)

In [ ]:
# 特徴量重要度を可視化してみる
fig, axes = plt.subplots(len(fimps_dfs), figsize=(15, 25))
for idx, fimps_df in enumerate(fimps_dfs):
    fimps_df.sort_values('1_importance').plot(kind='barh', ax=axes[idx])

* CryoSleepの一強っぽい？

# <div style="color: #F66;">感想・考察</div>

## 感想
* データの量　→ ちょうど良い、多すぎるわけではないので処理も軽い(ローカル環境で実行する可能性も考慮すると重要)
* 可視化 → カテゴリ、数値、テキストなどが混在しているので色々な手法を学ぶことができる
* 前処理 → 欠損値あり。ただ、偏りのある分布をしているので注意が必要。　ラベルエンコーディングもできる。 あとは特徴量の選択(有益じゃない特徴量を含まないなど)

<hr>

## 考察 
* `CryoSleep`が大事なのはわかった。
* 他に新しいカテゴリを作ることができるか思いつかない。一応案を載せておく
1. `Name`から性別フラグを作る
2. `Age`を区分で分ける
3. <s>`Spa`などの使用料金の項目を0・1以上・一定数以上で分ける</s>

* あとモデル作成が難しい。とりあえず`max_depth`は指定したけど、他の項目はどうしようか。しっかりデータの特徴を反映したものにしたい。
